# Zero Pressure Gradient Flat Plate

![plate](http://turbmodels.larc.nasa.gov/FlatPlate/Grids/plateBCpic.jpg)
    
#### References
    
http://turbmodels.larc.nasa.gov/flatplate.html

## Define case name
This is the solver case to be analysed

In [11]:
from collections import OrderedDict
case_dict = OrderedDict((   ('plate_medium',{ 'label' : '(68x48 cells)','face_area':0.075823,}),
                            ('plate_medium_f20',{'label' : '(68x48 cells)','face_area':0.075823,}),
                            ('plate_medium_f50',{'label' : '(68x48 cells)','face_area':0.075823,}),
                            ('plate_medium_f100',{'label' : '(68x48 cells)','face_area':0.075823,}),
                            ('plate_medium_f300',{'label' : '(68x48 cells)','face_area':0.075823,}),
                            ('plate_fine',{'label' : '(544x384 cells)','face_area':0.075823/8.0,})
                        ))

--------------------------------------------------------------------------
[dstandingford@login02] out: mpiexec noticed that process rank 1 with PID 13661 on node login02 exited on signal 11 (Segmentation fault).
[dstandingford@login02] out: --------------------------------------------------------------------------
[dstandingford@login02] out: 




Fatal error: run() received nonzero return code 139 while executing!

Requested: sleep 2;mpiexec -n 4 pvserver -rc --client-host=localhost -sp=12002
Executed: /bin/bash -l -c "cd /gpfs/thirdparty/zenotech/home/dstandingford/VALIDATION/PLATE && sleep 2;mpiexec -n 4 pvserver -rc --client-host=localhost -sp=12002"

None


Aborting.
run() received nonzero return code 139 while executing!

Requested: sleep 2;mpiexec -n 4 pvserver -rc --client-host=localhost -sp=12002
Executed: /bin/bash -l -c "cd /gpfs/thirdparty/zenotech/home/dstandingford/VALIDATION/PLATE && sleep 2;mpiexec -n 4 pvserver -rc --client-host=localhost -sp=12002"

None



## Define Data Location
For remote data the interaction will use ssh to securely interact with the data<br/>
This uses the reverse connection capability in paraview so that the paraview server can be submitted to a job scheduler<br/>
Note: The default paraview server connection will use port 11111

In [12]:
# from zutil.post import data_location_form_html
# data_location_form_html()

pvcluster_connect(data_host=data_host,data_dir=data_dir,paraview_cmd=paraview_cmd,job_queue=your_job_queue,
job_ntasks=your_number_of_mpi_tasks,job_ntaskpernode=your_number_of_tasks_per_node,job_project=your_job_project)

### Initialise Environment

In [ ]:
%pylab inline
from paraview.simple import *
paraview.simple._DisableFirstRenderCameraReset()
import pylab as pl
import math
import numpy as np

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Data Connection
This starts paraview server on remote host and connects

In [ ]:
from zutil.post import pvserver_connect
from zutil import post
if post.remote_data:
    pvserver_connect()
print 'Complete'

Testing passwordless ssh access


Get control dictionary

In [ ]:
from zutil.post import get_case_parameters,print_html_parameters
parameters={}
for case_name in case_dict:
    parameters[case_name] = get_case_parameters(case_name)
print 'Complete'

Get status file

In [ ]:
from zutil.post import get_status_dict
status=get_status_dict(case_name)
num_procs = str(status['num processor'])
print 'Complete'

### Define test conditions

In [ ]:
from IPython.display import HTML
for case_name in case_dict:
    HTML(print_html_parameters(parameters[case_name]))

In [ ]:
# Analysis constants
reference_area = 2.0
bl_position = 0.97

### Plotting functions

In [ ]:
from zutil.post import get_case_root, get_case_report
from zutil.post import get_csv_data,get_fw_csv_data
from zutil.post import for_each
def plot_theory(ax, filename):
    df = get_fw_csv_data(filename,widths=[16,16,16,16,16])
    ax.plot(df[1], df[0], color='grey', label='$u^{+}=y^{+}$')    
    ax.plot(df[2], df[0], color='grey', label='Log law')    

def plot_comparison(ax,filename):
    df = get_fw_csv_data(filename,widths=[16,14],skiprows=2)    
    ax.plot(df[0], df[1], color='red', label='CFL3D (545x385 points)')    
    
def velocity_plot(data,pts,**kwargs):
    ax = kwargs['axis']
    chart_label = kwargs['chart_label']
    ax.plot(data.GetPointData()['vprof'], pts.GetPoints()[:,2], label=chart_label)    

def plot_velocity_profile(ax,file_root,label,face_area):
    wall = PVDReader( FileName=file_root+'_wall.pvd' )
    
    print file_root+'_wall.pvd'
    
    drag = MinMax(Input=wall)
    drag.Operation = "SUM"

    drag.UpdatePipeline()
    
    drag_client = servermanager.Fetch(drag)
    cd = drag_client.GetCellData().GetArray("frictionforce").GetValue(0)
    
    wall_slice = Slice(Input=wall, SliceType="Plane" )
    
    wall_slice.SliceType.Normal = [1.0,0.0,0.0]
    wall_slice.SliceType.Origin = [bl_position, 0.0, 0.0]
    
    wall_slice.UpdatePipeline()
    
    wall_slice_client = servermanager.Fetch(wall_slice)
    nu = wall_slice_client.GetCellData().GetArray("nu").GetValue(0)
    utau = wall_slice_client.GetCellData().GetArray("ut").GetValue(0)
    yplus = wall_slice_client.GetCellData().GetArray("yplus").GetValue(0)
    cf    = wall_slice_client.GetCellData().GetArray("frictionforce").GetValue(0)
    wall_vel = wall_slice_client.GetCellData().GetArray("V").GetValue(0)
    
    wall_vel = 0.0

    symmetry = PVDReader( FileName=file_root+'_symmetry.pvd' )
    
    CellDatatoPointData1 = CellDatatoPointData(Input=symmetry)
    CellDatatoPointData1.PassCellData = 1
    
    # Removes second symmetry plane keeping y max
    Clip1 = Clip(Input=CellDatatoPointData1, ClipType="Plane" )
    Clip1.ClipType.Normal = [0.0,1.0,0.0]
    Clip1.ClipType.Origin = [0.0, -0.5, 0.0]
    
    Clip2 = Clip(Input=Clip1, ClipType="Plane" )
    Clip2.ClipType.Normal = [0.0,0.0,1.0]
    Clip2.ClipType.Origin = [0.0, 0.0, 0.05]
    Clip2.InsideOut = 1
    
    Slice1 = Slice(Input=Clip2, SliceType="Plane" )
    
    Slice1.SliceType.Normal = [1.0,0.0,0.0]
    Slice1.SliceType.Origin = [bl_position, 0.0, 0.0]
        
    Calculator2 = Calculator(Input=Slice1)
    
    Calculator2.AttributeMode = 'Point Data'
    Calculator2.Function = '(V.iHat - '+ str(wall_vel) +')'
    Calculator2.ResultArrayName = 'vprof'
    
    Calculator2.UpdatePipeline()
    
    sorted_line = PlotOnSortedLines(Input=Calculator2)
    sorted_line.UpdatePipeline()
    extract_client = servermanager.Fetch(sorted_line) 

    chart_label = '$y^{+}=' + ('%.2f$')%(yplus) + (' $C_d=%.6f$' % (cd/reference_area)) + (' $C_f=%.5f$' % (cf/face_area)) + ' ' + label

    for_each(extract_client,velocity_plot,axis=ax,chart_label=chart_label)
    
def bl_plot(data,pts,**kwargs):
    ax = kwargs['axis']
    chart_label = kwargs['chart_label']
    ax.plot(data.GetPointData()['yp'][1:], data.GetPointData()['up'][1:], label=chart_label)    

def plot_profile(ax,file_root,label):
    wall = PVDReader( FileName=file_root+'_wall.pvd' )
    
    wall_slice = Slice(Input=wall, SliceType="Plane" )
    
    wall_slice.SliceType.Normal = [1.0,0.0,0.0]
    wall_slice.SliceType.Origin = [bl_position, 0.0, 0.0]
    
    wall_slice.UpdatePipeline()
    
    wall_slice_client = servermanager.Fetch(wall_slice)
    nu = wall_slice_client.GetCellData().GetArray("nu").GetValue(0)
    #nu = 1.4e-5
    utau = wall_slice_client.GetCellData().GetArray("ut").GetValue(0)
    yplus = wall_slice_client.GetCellData().GetArray("yplus").GetValue(0)
    #roughness = wall_slice_client.GetCellData().GetArray("roughness").GetValue(0)
    #kplus = roughness*utau/nu
    
    t = wall_slice_client.GetCellData().GetArray("T").GetValue(0)
    p = wall_slice_client.GetCellData().GetArray("p").GetValue(0)
    rho = wall_slice_client.GetCellData().GetArray("rho").GetValue(0)
    
    wall_vel = wall_slice_client.GetCellData().GetArray("V").GetValue(0)
    
    symmetry = PVDReader( FileName=file_root+'_symmetry.pvd' )
    
    CellDatatoPointData1 = CellDatatoPointData(Input=symmetry)
    CellDatatoPointData1.PassCellData = 1
    
    # Removes second symmetry plane keeping y max
    Clip1 = Clip(Input=CellDatatoPointData1, ClipType="Plane" )
    Clip1.ClipType.Normal = [0.0,1.0,0.0]
    Clip1.ClipType.Origin = [0.0, -0.5, 0.0]
    
    Clip2 = Clip(Input=Clip1, ClipType="Plane" )
    Clip2.ClipType.Normal = [0.0,0.0,1.0]
    Clip2.ClipType.Origin = [0.0, 0.0, 0.05]
    Clip2.InsideOut = 1
    
    Slice1 = Slice(Input=Clip2, SliceType="Plane" )
    
    Slice1.SliceType.Normal = [1.0,0.0,0.0]
    Slice1.SliceType.Origin = [bl_position, 0.0, 0.0]
    
    Calculator1 = Calculator(Input=Slice1)
    
    #Calculator1.AttributeMode = 'point_data'
    Calculator1.AttributeMode = 'Point Data'
    Calculator1.Function = 'log10(coords.kHat * '+str(utau)+'/'+str(nu)+')'
    Calculator1.ResultArrayName = 'yp'
    
    Calculator2 = Calculator(Input=Calculator1)
    
    #Calculator2.AttributeMode = 'point_data'
    Calculator2.AttributeMode = 'Point Data'
    Calculator2.Function = '(V.iHat - '+ str(wall_vel) +')/ '+str(utau)
    Calculator2.ResultArrayName = 'up'
    
    Calculator2.UpdatePipeline()
    
    sorted_line = PlotOnSortedLines(Input=Calculator2)
    sorted_line.UpdatePipeline()
    extract_client = servermanager.Fetch(sorted_line) 

    chart_label = '$y^{+}='+('%.2f$')%(yplus)+' '+'$u_t='+('%.2f$')%(utau) + ' '+label

    for_each(extract_client,bl_plot,axis=ax,chart_label=chart_label)


## Plot Boundary Layer Profile

In [ ]:
from zutil.post import ProgressBar
pbar = ProgressBar()
fig = figure(figsize=(25, 10),dpi=100, facecolor='w', edgecolor='#E48B25')

ax = fig.add_subplot(1,2,1)
ax.grid(True)
ax.set_xlabel('$\mathbf{\log(y^{+})}$', fontsize=24, fontweight='bold', color = '#5D5858')
ax.set_ylabel('$\mathbf{u^{+}}$', fontsize=24, fontweight='bold', color = '#5D5858')
ax.set_title('Zero Pressure Gradient Flat Plate \n', fontsize=20, fontweight='normal', color = '#E48B25')
ax.axis([0,5,0,30])

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(18)
    tick.label.set_fontweight('normal') 
    tick.label.set_color('#E48B25')
    
plot_theory(ax,'data/u+y+theory.csv');
pbar+=5
plot_comparison(ax,'data/flatplate_u+y+_sst.dat');
pbar+=5
for case in case_dict:
    case_name = case
    label = case_dict[case]['label']
    plot_profile(ax,get_case_root(case_name,str(num_procs)),label)
    pbar+=5
# Draw legend
ax.legend(loc='lower right', shadow=True)
pbar+=5
# Create 
ax = fig.add_subplot(1,2,2)
ax.grid(True)
ax.set_xlabel('speed [m/s]', fontsize=16, fontweight='bold')
ax.set_ylabel('z', fontsize=16, fontweight='bold')
ax.set_title('Zero Pressure Gradient Flat Plate', fontsize=20, fontweight='bold')
#plot_comparison(ax,'data/flatplate_u_sst.dat');
#ax.axis([0,70,0,0.05])
pbar+=5
for case in case_dict:
    case_name = case
    label = case_dict[case]['label']
    plot_velocity_profile(ax,get_case_root(case_name,str(num_procs)),label,case_dict[case]['face_area'])
    pbar+=5
# Draw legend
ax.legend(loc='upper right', shadow=True)

fig.savefig("images/flat_plate_bl_profile.pdf")
pbar.complete()
show()
from IPython.display import FileLink, display 
display(FileLink('images/flat_plate_bl_profile.pdf')) 

## Mesh Convergence

In [ ]:
pbar = ProgressBar()
fig = figure(figsize=(10, 10),dpi=100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(1,1,1)
ax.grid(True)
ax.set_xlabel('$h=(1/N)^{1/2}$')
ax.set_ylabel('$C_f$ at $x=0.97$')
ax.set_title('Zero Pressure Gradient Flat Plate', fontsize=20, fontweight='bold')
df = get_fw_csv_data('data/cf_convergence_sst_fun3d.dat',widths=[8,13,12,21],skiprows=3) 
pbar+=25
ax.plot(df[2], df[3], color='red', marker='o',label='FUN3D')    
df = get_fw_csv_data('data/cf_convergence_sst_cfl3d.dat',widths=[8,13,12,21],skiprows=3) 
pbar+=25
ax.plot(df[2], df[3], color='blue', marker='o',label='CFL3D') 
df=[[1.75035e-2,2.18794e-3],[0.00261,0.00285]]
ax.plot(df[0], df[1], color='green', marker='o',label='zCFD')    
ax.legend(loc='upper right', shadow=True)

fig.savefig("images/flat_plate_mesh_conv_profile.pdf")
pbar.complete()
show()

from IPython.display import FileLink, display 
display(FileLink('images/flat_plate_mesh_conv_profile.pdf')) 

## Convergence

In [ ]:
from zutil.post import residual_plot, get_case_report
for case_name in case_dict:
    residual_plot(get_case_report(case_name))
show()

### Cleaning up

In [ ]:
if post.remote_data:
    #print 'Disconnecting from remote paraview server connection'
    #Disconnect()
    pass

<script type="text/javascript">
     show=true;
     function toggle(){
        if (show){
             $('div.input').hide();
         }else{
             $('div.input').show();
         }
         show = !show
     }
</script>
<a href="javascript:toggle()" target="_self">toggle input</a>